-donagram-1

In [1]:
import requests
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

# Global variable to keep track of the last update ID
last_update_id = None

# Function to send messages via Telegram
def send_telegram_message(message):
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'
    chat_id = '5255840420'
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
    response = requests.get(url)
    print(f"Sent message: {message}, Response: {response.json()}")  # Print the response from the Telegram API

# Function to request the next command from the user
def request_next_command():
    send_telegram_message("Please enter the next command (rerun, stop):")
    command = get_last_command(30)  # 30 second wait for a command
    while command not in ["rerun", "stop"]:
        if command:
            send_telegram_message("Invalid command. Please enter 'rerun' to rerun or 'stop' to stop training.")
        command = get_last_command(30)  # Wait another 30 seconds if invalid command received
    return command

# Function to check for new messages and wait for the next command with a timeout
def get_last_command(timeout=30):
    global last_update_id
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    start_time = time.time()
    print("Waiting for command...")

    while time.time() - start_time < timeout:
        response = requests.get(url).json()
        print(f"Checking for updates, Response: {response}")  # Debugging output
        try:
            updates = response['result']
            if updates:
                for update in reversed(updates):
                    update_id = update['update_id']
                    if 'message' in update and (last_update_id is None or update_id > last_update_id):
                        last_update_id = update_id
                        command = update['message']['text'].lower()
                        print(f"Received command: {command}")
                        return command
        except (IndexError, KeyError) as e:
            print(f"Error while processing updates: {e}")  # Debugging output
        time.sleep(1)  # Wait for 1 second before checking for new messages again
    return ""

# Define the model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

# Setup for training
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(num_epochs, train_loader, model, criterion, optimizer):
    model.train()
    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in enumerate(train_loader):
            outputs = model(data)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    return loss.item()

# Main loop
while True:
    send_telegram_message("Model training has started.")
    final_loss = train_model(1, train_loader, model, criterion, optimizer)
    send_telegram_message(f"Training complete. Final loss: {final_loss:.4f}")

    command = request_next_command()
    if command == "stop":
        send_telegram_message("Training stopped by user command.")
        break  # Exit loop if the received command is 'stop'
    elif command == "rerun":
        send_telegram_message("Rerunning the training.")
        continue  # Continue the loop to rerun the training


c:\Users\smart\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\smart\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 20562816.16it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 950538.64it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 6519413.28it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4510068.36it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Sent message: Model training has started., Response: {'ok': True, 'result': {'message_id': 103, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721805755, 'text': 'Model training has started.'}}
Epoch 1, Loss: 0.08431698381900787
Sent message: Training complete. Final loss: 0.0843, Response: {'ok': True, 'result': {'message_id': 104, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721805838, 'text': 'Training complete. Final loss: 0.0843'}}
Sent message: Please enter the next command (rerun, stop):, Response: {'ok': True, 'result': {'message_id': 105, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anora

-donagram-2

In [4]:
import requests
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


last_update_id = None


def send_telegram_message(message):
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'
    chat_id = '5255840420'
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
    response = requests.get(url)
    print(f"Sent message: {message}, Response: {response.json()}")


def send_telegram_image(image_path):
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'
    chat_id = '5255840420'
    url = f"https://api.telegram.org/bot{TOKEN}/sendPhoto"
    with open(image_path, 'rb') as image_file:
        files = {'photo': image_file}
        data = {'chat_id': chat_id}
        response = requests.post(url, files=files, data=data)
    print(f"Sent image: {image_path}, Response: {response.json()}")

def request_next_command():
    send_telegram_message("Please enter the next command (rerun, stop):")
    command = get_last_command(30)  
    while command not in ["rerun", "stop"]:
        if command:send_telegram_message("Invalid command. Please enter 'rerun' to rerun or 'stop' to stop training.")
        command = get_last_command(30) 
    return command


def get_last_command(timeout=30):
    global last_update_id
    TOKEN = '7166875544:AAGBi7azOdXowUbrZKXHpm7p152pG-mDxkA'
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    start_time = time.time()
    print("Waiting for command...")
    while time.time() - start_time < timeout:
        response = requests.get(url).json()
        print(f"Checking for updates, Response: {response}") 
        try:
            updates = response['result']
            if updates:
                for update in reversed(updates):
                    update_id = update['update_id']
                    if 'message' in update and (last_update_id is None or update_id > last_update_id):
                        last_update_id = update_id
                        command = update['message']['text'].lower()
                        print(f"Received command: {command}")
                        return command
        except (IndexError, KeyError) as e:
            print(f"Error while processing updates: {e}") 
        time.sleep(1) 
    return ""

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512),nn.ReLU(),nn.Linear(512, 10))

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(num_epochs, train_loader, model, criterion, optimizer):
    model.train()
    epoch_losses = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_idx, (data, targets) in enumerate(train_loader):
            outputs = model(data)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        epoch_loss /= len(train_loader)
        epoch_losses.append(epoch_loss)
        print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')
    return epoch_losses

while True:
    send_telegram_message("Model training has started.")
    epoch_losses = train_model(2, train_loader, model, criterion, optimizer)
    final_loss = epoch_losses[-1]
    send_telegram_message(f"Training complete. Final loss: {final_loss:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(epoch_losses, marker='o', linestyle='-', color='b')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig('training_loss_plot.png')
    plt.close()  

    send_telegram_image('training_loss_plot.png')

    command = request_next_command()
    if command == "stop":
        send_telegram_message("Training stopped by user command.")
        break 
    elif command == "rerun":
        send_telegram_message("Rerunning the training.")
        continue 

Sent message: Model training has started., Response: {'ok': True, 'result': {'message_id': 150, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721906181, 'text': 'Model training has started.'}}
Epoch 1, Loss: 0.3085
Epoch 2, Loss: 0.1389
Sent message: Training complete. Final loss: 0.1389, Response: {'ok': True, 'result': {'message_id': 151, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 1721906295, 'text': 'Training complete. Final loss: 0.1389'}}
Sent image: training_loss_plot.png, Response: {'ok': True, 'result': {'message_id': 152, 'from': {'id': 7166875544, 'is_bot': True, 'first_name': 'anorak', 'username': 'gladiator_anorak_bot'}, 'chat': {'id': 5255840420, 'first_name': 'Anorak', 'type': 'private'}, 'date': 17